# Assignment 4: Evaluating Search Engines

For this assignment, we leave aside the code we developed so far, and look into the more general issue of how to evaluate and compare different search engines. The ultimate test for any Information Retrieval system is how well it is able to satisfy the information needs of users.

# Cohen's Kappa

Our evaluation will involve the calculation of [Cohen's Kappa](https://en.wikipedia.org/wiki/Cohen's_kappa) to quantify the degree to which two human assessors agree or disagree on whether results are considered relevant or not. To calculate Cohen's Kappa, we are going to use the [scikit-learn library](http://scikit-learn.org/stable/):

In [1]:
! pip install --user scikit-learn

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from sklearn.metrics import cohen_kappa_score

This library expects relevance assessments as lists of elements where `1` stands for _relevant_ and `0` stands for _not relevant_, for example like this:

In [3]:
a1=[1,0,1,0,1,0,1,0]

This list means that the first document was assessed to be relevant, the second to be not relevant, the third to be relevant etc.

We need two assessments in order to calculate Cohen's Kappa, so let's make another exemplary list that only differs on the last element:

In [4]:
a2=[1,0,1,0,1,0,1,1]

We can now invoke the library as follows to calculate the agreement between the two:

In [5]:
cohen_kappa_score(a1, a2)

0.75

This value represents high agreement. We can reach maximal agreement if the two assessments are identical:

In [6]:
cohen_kappa_score(a1, a1)

1.0

Now, let's see what happens for a third assessment that differs on three positions with the first one (the three last positions):

In [7]:
a3=[1,0,1,0,1,1,0,1]

cohen_kappa_score(a1, a3)

0.25

We get a smaller but still positive value, because these two assessments still mostly agree. If we make a further example that differs on 6 of the 8 positions, we get the following result:

In [8]:
a4=[1,0,0,1,0,1,0,1]

cohen_kappa_score(a1, a4)

-0.5

The score is now negative, because the two differ on more positions than they agree. The agreement is in fact less than what you would expect to occur just by chance. We get the maximal disagreement if we define a fifth example that disagrees on all positions:

In [9]:
a5=[0,1,0,1,0,1,0,1]

cohen_kappa_score(a1, a5)

-1.0

Now that we understand how this function works, we will apply it below for our specific evaluation.

# Results and Assessments

Next, we will define some auxilary code to deal with lists of URLs from search engines and associated relevance assessments. We will encode result lists like this:

In [10]:
urls = [
    'https://en.wikipedia.org/wiki/Information_retrieval/',  # 1st result
    'http://www.dictionary.com/browse/information',          # 2nd result
    'https://nlp.stanford.edu/IR-book/'                      # ...
]

And we represent corresponding assessments, as above, as lists of the same size containing relevance values:

In [11]:
my_assessment = [1, 0, 1]
another_assessment = [0, 0, 1]

In order to nicely display URL lists, with or without related assessments, we define a function called `display_results`:

In [12]:
from IPython.display import display, HTML

def display_results(urls, assessment1=None, assessment2=None):
    lines = []
    lines.append('<table>')
    header = '<tr><th>#</th><th>Result URL</th>'
    if (assessment1):
        header += '<th>Assessment 1</th>'
    if (assessment2):
        header += '<th>Assessment 2</th>'
    header += '</tr>'
    lines.append(header)
    i = 0
    for url in urls:
        show_url = url
        if (len(url) > 80):
            show_url = url[:75] + '...'
        line = '<tr><td>{}</td><td><a href="{:s}">{:s}</a></td>'.format(i+1, url, show_url)
        if (assessment1):
            if (assessment1[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        if (assessment2):
            if (assessment2[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        line += '</tr>'
        lines.append(line)
        i = i+1
    lines.append('</table>')
    display( HTML(''.join(lines)) )

We can use this function to display a list of URLs, optionally together with one or two assessment lists:

In [13]:
print("Just a list of URLs:")
display_results(urls)

print("With one assessment:")
display_results(urls, my_assessment)

print("With two assessments:")
display_results(urls, my_assessment, another_assessment)

Just a list of URLs:


#,Result URL
1,https://en.wikipedia.org/wiki/Information_retrieval/
2,http://www.dictionary.com/browse/information
3,https://nlp.stanford.edu/IR-book/


With one assessment:


#,Result URL,Assessment 1
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant
2,http://www.dictionary.com/browse/information,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant


With two assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant,Not relevant
2,http://www.dictionary.com/browse/information,Not relevant,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant,Relevant


Now we are ready to perform an actual evaluation, which will involve a substantial amount of manual work.

---

# Tasks

**Your name:** Mai Nguyen

### Task 1

Think up and formulate an information need in the areas of Computer Science or the Life Sciences (medicine, biology, etc.) for which you think the answer can be found in scientific publications. On page 152 in the book an example of such an information need is shown: "Information on whether drinking red wine is more effective at reducing your risk of heart attacks than white wine."

**Answer:** How is machine learning applied in the field of healthcare informatics?

Next, write down specifically what documents have to look like to satisfy your information need. For example if your information need is about finding an overview of different cancer types, you could state that a document would need to list at least ten types of cancer to satisfy your information need (among other criteria). Write this down as a protocol with rules and examples. For example, such a protocol could state that at least three out of five given criteria have to be fulfilled for a document to be considered relevant for the information need, and then specify the criteria. Or your protocol could have the form of a sequence of rules, where each rule lets you either label the document as relevant or not relevant, or proceed with the next rule. Such rules and criteria can, for example, be about the general topic of the paper, the concepts mentioned in it, the covered relations between concepts, the type of publication (research paper, overview paper, etc.), the number of references, the types of contained diagrams, and so on, depending on your specified information need.

**Answer:** A document is considered relevant when it satifies **ALL** of the criteria below:
1. The main topic of the paper is concerned with machine learning and healthcare informatics
2. The paper is published in a trusted scientific journal
3. The number of references is at least 20
4. The paper has to mention at least 1 application of machine learning in healthcare informatics, point out what machine learning algorithms are used and discuss how it is implemented to resolve a problem or help improve a situation in healthcare informatics
5. The paper has to include a study/experiment investigating whether the use of machine learning betters a healthcare informatics-related issue

### Task 2

Formulate a keyword query that represents the information need. For the example on page 152 in the book (see above), the example query "wine AND red AND white AND heart AND attack AND effective" is given. (You don't need to use connectors like "AND", but if you do, make first sure your chosen search engines below actually support them.)

**Answer:** machine learning healthcare informatics

Then submit your query to **two** of the following academic search engines:

- [Google Scholar](https://scholar.google.com) (all science disciplines)
- [Semantic Scholar](https://www.semanticscholar.org) (computer science and biomedicine)
- [PubMed Search](https://www.ncbi.nlm.nih.gov/pubmed) (Life Sciences / biomedicine)

The right choice of two from the three search engine depends on the topic of your information need. If your information need is in Computer Science, for example, you should use Google Scholar and Semantic Scholar.

Extract a list of the top 10 URLs of the lists of each of the search engines
given the query. Try to access the resulting publications. For the publications
where that is not possible (because of dead links or because the publication is
pay-walled even within the VU network), exclude them from the list and add more publications to the end of
your list (that is, append results number 11, then 12, etc. to ensure you have
two lists of 10 publications each). In order to deal with paywalls, you should try accessing the articles from the VU network, use
[UBVU Off-Campus
Access](http://www.ub.vu.nl.vu-nl.idm.oclc.org/nl/faciliteiten/toegang-buiten-de-campus/index.aspx), or try to find the respective documents from alternative sources (Google Scholar, for example, is very good at finding free PDFs of articles). If you get fewer than 10 results for one of the search engines, modify the keyword query above to make it more inclusive, and then redo the steps of this task.

Store your two lists of URLs in the form of Python lists as introduced above. Then, use the `display_results` function to nicely display them.

In [14]:
# Create two of the lists below, depending on your chosen engines:

urls_google = ['https://link.springer.com/article/10.1007/S40708-016-0042-6',
              'https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.22147',
              'https://graz.pure.elsevier.com/de/publications/trends-in-interactive-knowledge-discovery-for-personalized-medici',
              'https://academic.oup.com/jamia/article/18/5/552/830538',
              'https://www.healthaffairs.org/doi/abs/10.1377/hlthaff.2014.0053',
              'https://ieeexplore.ieee.org/iel7/6221020/6363502/07801947.pdf',
              'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0030412',
              'https://www.sciencedirect.com/science/article/pii/S1532046403000856',
              'https://academic.oup.com/jamia/article/18/4/498/2909142',
              'https://academic.oup.com/jamia/article/20/5/859/727909']
urls_semantic = ['https://www.semanticscholar.org/paper/Big-data-analytics-in-healthcare%3A-promise-and-Raghupathi-Raghupathi/3895912b187adee599b1ea662da92865dd0b197d',
                'https://www.semanticscholar.org/paper/Clinical-Quality-Needs-Complex-Adaptive-Systems-and-Marsland-Buchan/366b666d7f0cccfec7e035e31a1dc2976144d6a6',
                'https://www.semanticscholar.org/paper/Machine-Learning-for-Health-Informatics-Holzinger/8605b2200a46b2484a116a02f8dc8ff1bdcf0802',
                'https://www.semanticscholar.org/paper/A-Prototype-Agent-Based-Model-and-Machine-Learning-Laskowski/c19194b86d66886c7e454ff7622a4551ecf1dfab',
                'https://www.semanticscholar.org/paper/Health-Informatics-via-Machine-Learning-for-the-of-Clifton-Niehaus/5a8db2b6866f12ded696c54ed22549bd7896bbb0',
                'https://www.semanticscholar.org/paper/Beyond-Volume%3A-The-Impact-of-Complex-Healthcare-on-Feldman-Faust/cd0751966d473e6ddd63fdd3d7668fd877e4d6e7',
                'https://www.semanticscholar.org/paper/How-can-machine-learning-methods-assist-in-virtual-Ichikawa-Saito/8c08a1b7d89d217a91a98088ebfe13d94db09f10',
                'https://www.semanticscholar.org/paper/Translational-bioinformatics-and-healthcare-and-Sethi-Theodos/3f733d7fd1801674126dcd94b74e00ebbcb36674',
                'https://www.semanticscholar.org/paper/A-comparative-analysis-of-machine-learning-methods-Emanet-%C3%96z/df0b6f09296330c955fa6acc9770f904d79b614a',
                'https://www.semanticscholar.org/paper/The-Learning-Healthcare-System-and-Cardiovascular-Maddox-Borden/fef8ba37718c52fc67524861b09caf121cb3c22e']

# Call display_results here
display_results(urls_google)
display_results(urls_semantic)

#,Result URL
1,https://link.springer.com/article/10.1007/S40708-016-0042-6
2,https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.22147
3,https://graz.pure.elsevier.com/de/publications/trends-in-interactive-knowle...
4,https://academic.oup.com/jamia/article/18/5/552/830538
5,https://www.healthaffairs.org/doi/abs/10.1377/hlthaff.2014.0053
6,https://ieeexplore.ieee.org/iel7/6221020/6363502/07801947.pdf
7,https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0030412
8,https://www.sciencedirect.com/science/article/pii/S1532046403000856
9,https://academic.oup.com/jamia/article/18/4/498/2909142
10,https://academic.oup.com/jamia/article/20/5/859/727909


#,Result URL
1,https://www.semanticscholar.org/paper/Big-data-analytics-in-healthcare%3A-p...
2,https://www.semanticscholar.org/paper/Clinical-Quality-Needs-Complex-Adapti...
3,https://www.semanticscholar.org/paper/Machine-Learning-for-Health-Informati...
4,https://www.semanticscholar.org/paper/A-Prototype-Agent-Based-Model-and-Mac...
5,https://www.semanticscholar.org/paper/Health-Informatics-via-Machine-Learni...
6,https://www.semanticscholar.org/paper/Beyond-Volume%3A-The-Impact-of-Comple...
7,https://www.semanticscholar.org/paper/How-can-machine-learning-methods-assi...
8,https://www.semanticscholar.org/paper/Translational-bioinformatics-and-heal...
9,https://www.semanticscholar.org/paper/A-comparative-analysis-of-machine-lea...
10,https://www.semanticscholar.org/paper/The-Learning-Healthcare-System-and-Ca...


### Task 3

Then, find a fellow student who will **independently**
assess the results as "relevant" or "not relevant" using the protocol that you
have defined above, and also help (at least) one other student for his/her
assessment. Write down their names here:

**Name of the student who assesses my results:** Markus Moosbrugger

**Name of the student who I help to assess his/her results:** Sara Real

Show to the other assessor everything you have written down above for Tasks 1 and 2 (and you might also want to give him/her the PDFs you got for these papers to simplify the process).

You as assessors need to stick to the protocol you made in Task 1 and should not discuss with each other, especially when you doubt whether a result is relevant or not. Write down your assessments as lists of relevance values, as introduced above, and make sure they correctly map to the URLs by displaying them together with the `display_results` function.

To avoid problems with extreme results, mark in each list at least one paper as 'relevant' and at least one paper as 'not relevant'. That is, if all papers seem relevant, mark the one that seems least relevant 'not relevant', and conversely, if none of the papers seem relevant, mark the one that seems a bit more relevant than the others as 'relevant'.

In [15]:
# 0 = not relevant; 1 = relevant

# You only need to create 4 of the following 6 lists, again depending on which search engines you chose.

# Assessment 1 is from you:

assessment1_google = [1,1,0,1,0,1,1,0,1,1]
assessment1_semantic = [0,0,0,1,0,0,1,0,1,0] 


# Assessment 2 is from your fellow student (don't show him/her your own assessment!):

assessment2_google = [1,1,0,0,0,1,1,0,1,1]
assessment2_semantic = [0,1,1,0,0,0,1,0,1,0]


# Call display_results here
display_results(urls_google,assessment1_google,assessment2_google)
display_results(urls_semantic,assessment1_semantic,assessment2_semantic)

#,Result URL,Assessment 1,Assessment 2
1,https://link.springer.com/article/10.1007/S40708-016-0042-6,Relevant,Relevant
2,https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.22147,Relevant,Relevant
3,https://graz.pure.elsevier.com/de/publications/trends-in-interactive-knowle...,Not relevant,Not relevant
4,https://academic.oup.com/jamia/article/18/5/552/830538,Relevant,Not relevant
5,https://www.healthaffairs.org/doi/abs/10.1377/hlthaff.2014.0053,Not relevant,Not relevant
6,https://ieeexplore.ieee.org/iel7/6221020/6363502/07801947.pdf,Relevant,Relevant
7,https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0030412,Relevant,Relevant
8,https://www.sciencedirect.com/science/article/pii/S1532046403000856,Not relevant,Not relevant
9,https://academic.oup.com/jamia/article/18/4/498/2909142,Relevant,Relevant
10,https://academic.oup.com/jamia/article/20/5/859/727909,Relevant,Relevant


#,Result URL,Assessment 1,Assessment 2
1,https://www.semanticscholar.org/paper/Big-data-analytics-in-healthcare%3A-p...,Not relevant,Not relevant
2,https://www.semanticscholar.org/paper/Clinical-Quality-Needs-Complex-Adapti...,Not relevant,Relevant
3,https://www.semanticscholar.org/paper/Machine-Learning-for-Health-Informati...,Not relevant,Relevant
4,https://www.semanticscholar.org/paper/A-Prototype-Agent-Based-Model-and-Mac...,Relevant,Not relevant
5,https://www.semanticscholar.org/paper/Health-Informatics-via-Machine-Learni...,Not relevant,Not relevant
6,https://www.semanticscholar.org/paper/Beyond-Volume%3A-The-Impact-of-Comple...,Not relevant,Not relevant
7,https://www.semanticscholar.org/paper/How-can-machine-learning-methods-assi...,Relevant,Relevant
8,https://www.semanticscholar.org/paper/Translational-bioinformatics-and-heal...,Not relevant,Not relevant
9,https://www.semanticscholar.org/paper/A-comparative-analysis-of-machine-lea...,Relevant,Relevant
10,https://www.semanticscholar.org/paper/The-Learning-Healthcare-System-and-Ca...,Not relevant,Not relevant


### Task 4

Compute Cohen's kappa to quantify how much the two assessors agreed. Use the function `cohen_kappa_score` demonstrated above to calculate two times the inter-annotator agreement (once for each of the two search engines), and print out the results.

In [16]:
# Add your code here:

kappa_google = cohen_kappa_score(assessment1_google,assessment2_google)
kappa_semantic = cohen_kappa_score(assessment1_semantic,assessment2_semantic)

print("Kappa for Google Scholar:", kappa_google)
print("Kappa for Semantic Scholar:", kappa_semantic)

Kappa for Google Scholar: 0.7826086956521738
Kappa for Semantic Scholar: 0.34782608695652173


Explain whether the agreement can be considered high or not, based on the interpretation table on [this Wikipedia page](https://en.wikipedia.org/wiki/Fleiss'_kappa#Interpretation) (this Wikipedia page is about a different type of kappa but the interpretation table can also be used for Cohen's kappa).

**Answer:** Based on the interpretation table, there is a substantial agreement in regard to the relevance of Google Scholar results, whereas there is only a fair agreement on the relevance of Semantic Scholar results.

### Task 5

Define a function called `precision_at_n` that calculates Precision@n as described in the lecture slides, which takes as input an assessment list and a value for _n_ and returns the respective Precision@n value. Run this function to calculate Precision@10 (that is, n=10) on all four assessments (two assessors and two search engines).

In [17]:
#Add your code here:

def precision_at_n(assessment_list, n):
    return sum(assessment_list)/n

# Print out Precision@10 for all assessments here.
print("Precision@10 for my assessment of Google Scholar results: ",precision_at_n(assessment1_google,10))
print("Precision@10 for my peer's assessment of Google Scholar results: ",precision_at_n(assessment2_google,10))
print("Precision@10 for my assessment of Semantic Scholar results: ",precision_at_n(assessment1_semantic,10))
print("Precision@10 for my peer's assessment of Semantic Scholar results: ",precision_at_n(assessment2_semantic,10))

Precision@10 for my assessment of Google Scholar results:  0.7
Precision@10 for my peer's assessment of Google Scholar results:  0.6
Precision@10 for my assessment of Semantic Scholar results:  0.3
Precision@10 for my peer's assessment of Semantic Scholar results:  0.4


Explain what these specific Precision@10 results tell us (or don't tell us) about the quality of the two search engines for your particular information need. You can also refer to the results of Task 4 if necessary.

**Answer:** Google Scholar has much higher precision value than Semantic Scholar. In other words, Google Scholar yields more relevant results than Semantic Scholar does for my information need.Therefore, Google Scholar is of better quality for my information need than Semantic Scholar.

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.